In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import os, math

In [3]:
def plot(algos, regrets, cumsum=False, filename=None, figshow=True, legendloc='bottom left'):
    assert len(algos) == len(regrets)
    num_algos = len(algos)
    
    # Get color range
    cmap = plt.cm.get_cmap('jet')
    colors = cmap(np.arange(cmap.N))
    cinv = cmap.N / (1. * num_algos) 
    
    
    if cumsum:
        regrets_ = []
        for r in regrets:
            regrets_.append(r.cumsum(axis=1))
        regrets = regrets_
    
    # Compute mu and std 
    mus = []
    stds = []
    for r in regrets:
        mu = np.mean(r,axis=0)
        std = np.std(r,axis=0) / math.sqrt(r.shape[0])
        mus.append(mu)
        stds.append(std) 
        
    # Plot 
    T = regrets[0].shape[1]
    fig = plt.figure(figsize=(5,4))
    dom = np.arange(T)

    for i in range(num_algos):
        color = colors[int( (i + 0.5) * cinv)] 
        plt.plot(dom, mus[i], label=algos[i], color=color) 
        plt.fill_between(dom, mus[i] + 1.96 * stds[i],\
                         mus[i] - 1.96 * stds[i], alpha=0.2, color=color)

    plt.xlim([0,T])
    plt.rc('grid', linestyle='--', color='black', alpha=0.2)
    plt.grid('on')
    
    if legendloc:
        plt.legend(loc = legendloc, prop={'size': 11}) 
    else:
        plt.legend(prop={'size':11})


    dom = np.arange(0, T, 200)
    plt.xticks(dom, dom*10)
    plt.xlabel('Number of samples',fontsize=12)
    plt.ylabel('Sub-optimality',fontsize=12)

    if filename:
        if not os.path.exists('fig'):
            os.makedirs('fig')
        fig.savefig(os.path.join('fig', filename), dpi=500, bbox_inches='tight')
    
    if not figshow:
        plt.close(fig)
        plt.clf()

In [15]:
def plot_realworld_experiment(name, trainconfig, beta, rbf_sigma, lr=0.001, figshow=False, legendloc=None, save=True):
    res_dir = os.path.join('results', name)
    approx_neural = 'approx-neural-gridsearch_epochs={}_m=100_layern=True_buffer={}_bs={}_lr={}_beta={}_lambda=0.0001_lambda0=0.1.npz'.format(trainconfig[0],trainconfig[1],trainconfig[2],lr,beta)
    neural_greedy = 'neural-greedy-gridsearch_epochs={}_m=100_layern=True_buffer={}_bs={}_lr={}_lambda=0.0001.npz'.format(trainconfig[0],trainconfig[1],trainconfig[2],lr)
    baseline = 'baseline_epochs=100_m=100_layern=True_beta=0.1_lambda0=0.1_rbf-sigma=1.0_maxnum=1000.npz'
    kern = 'kern-gridsearch_beta=0.1_rbf-sigma={}_maxnum=1000.npz'.format(rbf_sigma)

    neuralcb = np.load(os.path.join(res_dir,approx_neural))['arr_0']
    neuralgreedy = np.load(os.path.join(res_dir,neural_greedy))['arr_0']
    bl = np.load(os.path.join(res_dir,baseline))['arr_0']
    kernlcb = np.load(os.path.join(res_dir,kern))['arr_0']

    # Read data 
    arrs = (neuralcb, neuralgreedy, bl, kernlcb)
    algos = ['NeuraLCB', 'NeuralGreedy', 'LinLCB', 'NeuralLinLCB', 'NeuralLinGreedy', 'KernLCB']
    regrets = [] 
    i = 0
    for a in arrs:
        for i in range(1,a.shape[1]):
            regrets.append(a[:,i,:])

    # Plot
    filename = '{}_cf={}{}{}_beta={}_lr={}_rbfsigma={}.pdf'.format(name.split('_')[0],trainconfig[0],trainconfig[1],trainconfig[2],beta,lr,rbf_sigma)
    print(filename)
    plot(algos, regrets, cumsum=False, filename=filename if save else None, figshow=figshow, legendloc=legendloc)

In [19]:
data_types = [
             'mushroom_d=117_a=2_pi=eps-greedy0.1_std=0']
              #'statlog_d=10_a=7_pi=eps-greedy0.1_std=0',
              #'stock_d=21_a=8_pi=eps-greedy0.1_std=0.1',
              #'jester_d=32_a=8_pi=eps-greedy0.1_std=0.1', 
              #'covertype_d=54_a=7_pi=eps-greedy0.1_std=0', 
              #'census_d=387_a=9_pi=eps-greedy0.1_std=0.0',
              #'adult_d=92_a=14_pi=eps-greedy0.1_std=0.0']
betas = [0.01,0.05,0.1,1.0,5.0, 2.0, 10.0]
trconfigs =  [(1,1,1),(100,-1,50),(30,100,16)]
rbf_sigmas = [0.1,1.0,10.0]
lrs = [0.001,0.0001]
for data_type in data_types:
    for trconfig in trconfigs:
        for beta in betas:
            for lr in lrs:
                for rbf_sigma in rbf_sigmas:
                    try:
                        plot_realworld_experiment(data_type, trconfig, beta, rbf_sigma, lr)
                    except: 
                         pass

mushroom_cf=111_beta=5.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=111_beta=5.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=111_beta=5.0_lr=0.0001_rbfsigma=10.0.pdf
mushroom_cf=111_beta=2.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=111_beta=2.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=111_beta=2.0_lr=0.0001_rbfsigma=10.0.pdf
mushroom_cf=111_beta=10.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=111_beta=10.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=111_beta=10.0_lr=0.0001_rbfsigma=10.0.pdf
mushroom_cf=3010016_beta=5.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=3010016_beta=5.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=3010016_beta=5.0_lr=0.0001_rbfsigma=10.0.pdf
mushroom_cf=3010016_beta=2.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=3010016_beta=2.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=3010016_beta=2.0_lr=0.0001_rbfsigma=10.0.pdf
mushroom_cf=3010016_beta=10.0_lr=0.0001_rbfsigma=0.1.pdf
mushroom_cf=3010016_beta=10.0_lr=0.0001_rbfsigma=1.0.pdf
mushroom_cf=3010016_beta=10.0_lr=0.0001_rbfsigma=10.0.pdf


<Figure size 432x288 with 0 Axes>